In [62]:
####Mixture Multinomial Logit Model####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
import scipy.stats as ss
from numpy.random import *
from scipy import optimize
from scipy.stats import norm
from scipy import sparse

In [63]:
####データの発生####
##データの設定
segment = 5
hh = 10000   #サンプル数
pt = numpy.random.poisson(numpy.random.gamma(5.0, 1/0.5, hh))   #購買機会数
pt[pt==0] = 1   #購買機会が0なら1に置き換え
hhpt = np.sum(pt)   #総サンプル数
member = 10   #選択可能数
k = 5   #説明変数数

In [64]:
##idとセグメントの設定
#idの設定
id = np.repeat(range(hhpt), member)
no = np.zeros(hhpt*member, dtype='int')
start = 0
for i in range(hhpt):
    no[start:start+member] = np.array(range(member))
    start += member

user_id = np.repeat(range(hh), pt)
pt_id = np.zeros(hhpt, dtype='int')
start = 0
for i in range(hh):
    pt_id[start:start+pt[i]] = np.array(range(pt[i]))
    start += pt[i]
u_id = np.repeat(user_id, member)
user_dt = sparse.coo_matrix((np.repeat(1, hhpt), (range(hhpt), user_id)), shape=(hhpt, hh)).T

In [65]:
#インデックスを設定
index = np.array(range(hhpt*member))
user_list = [i for i in range(hh)]
id_list = [i for i in range(hhpt)]
no_list = [i for i in range(member)]

for i in range(hh):
    user_list[i] = index[u_id==i]
for i in range(hhpt):
    id_list[i] = index[id==i]
for j in range(member):
    no_list[j] = index[no==j]

In [66]:
#セグメントの設定
theta = thetat = np.random.dirichlet(np.repeat(3.0, segment), 1).reshape(-1)
Z = np.random.multinomial(1, theta, hh)
seg = np.dot(Z, range(segment))

In [67]:
##説明変数の発生
#切片の設定
diag_ind = np.ravel(np.diag(np.ravel(np.matlib.repmat(1, member, 1))))
diag_vec = np.ravel(np.matlib.repmat(diag_ind, hhpt, 1))
intercept = np.reshape(diag_vec, (hhpt*member, member))[:, range(member-1)]

In [68]:
#衣装の割当を設定
c_num = 8
prob = np.random.dirichlet(np.repeat(1.5, c_num), member); m = np.argmin(np.sum(prob, axis=0))
Cloth = np.zeros((hhpt*member, c_num-1))
for j in range(member):
    x = np.random.multinomial(1, prob[j, ], hhpt)
    Cloth[no_list[j], :] = np.delete(x, m, axis=1)

In [69]:
#どのメンバーの勧誘回だったか
prob = np.repeat(1/member, member)
scout = np.zeros((hhpt, member))
for i in range(hhpt):
    while True:
        x = np.random.multinomial(2, prob, 1).reshape(-1)
        if np.max(x) == 1:
            break
    scout[i, :] = x
Scout = scout.reshape(-1)

In [70]:
#レベルの対数
lv_weibull = np.round(scipy.stats.weibull_min.rvs(1.8, scale=250, size=hh*member))
lv = np.log(np.random.choice(lv_weibull[lv_weibull > 80], hh, replace=False))
Lv = np.zeros((hhpt*member, member-1))
for i in range(hh):
    Lv[user_list[i], :] = (lv[i] * np.identity(member))[:, range(member-1)][np.tile(range(member), pt[i]), :]

In [71]:
#スコアの対数
score = np.abs(np.random.normal(0, 0.75, hhpt))
Score = np.zeros((hhpt*member, member-1))
for i in range(hh):
    Score[user_list[i], :] = (score[i] * np.identity(member))[:, range(member-1)][np.tile(range(member), pt[i]), :]

In [72]:
#データの結合
Data = np.hstack((intercept, Cloth, Scout.reshape((hhpt*member, 1)), Lv, Score))
sparse_data = scipy.sparse.csr_matrix(Data)
k1 = intercept.shape[1]; k2 = Cloth.shape[1]; k3 = 1; k4 = Lv.shape[1]; k5 = Score.shape[1]
k = Data.shape[1]
pd.DataFrame(np.round(Data, 3))   #データを確認

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.103,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.000,0.000,0.103,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.103,0.000,0.000,0.000,0.000,0.000,0.000
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.103,0.000,0.000,0.000,0.000,0.000
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.000,0.000,0.000,0.000,0.000,0.103,0.000,0.000,0.000,0.000
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.103,0.000,0.000,0.000
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.103,0.000,0.000
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.103,0.000
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,4.644,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.103
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [73]:
##パラメータの設定
beta1 = beta4 = beta5 = np.zeros((segment, member-1))
beta2 = np.zeros((segment, c_num-1))
beta3 = np.repeat(0.0, segment)

for j in range(segment):
    beta1[j, :] = np.random.uniform(-1.0, 4.0, member-1)   #切片の回帰係数
    beta2[j, ] = np.random.uniform(-2.0, 3.0, c_num-1)   #衣装の回帰係数
    beta3[j] = np.random.uniform(0.6, 4.0, 1)   #勧誘の回帰係数
    beta4[j, :] = np.random.uniform(-0.4, 0.4, member-1)   #レベルの回帰係数
    beta5[j, :] = np.random.uniform(-0.6, 0.6, member-1)   #スコアの回帰係数
beta = betat = np.hstack((beta1, beta2, beta3.reshape((segment, 1)), beta4, beta5)).T   #回帰係数を結合
b = beta.reshape(-1)

In [74]:
##応答変数の発生
#ロジットと確率を発生
U = np.sum(np.dot(Data, beta) * Z[u_id, ], axis=1).reshape(hhpt, member)
Pr = np.exp(U) / np.sum(np.exp(U), axis=1).reshape(hhpt, 1)

In [75]:
#応答変数の発生
y = np.array([np.random.multinomial(1, Pr[i, :], 1) for i in range(hhpt)]).reshape((hhpt, member))
y_vec = y.reshape(-1)
np.sum(y, axis=0)

array([ 3855,  5701, 16493,  7735,  8802,  8226, 31632,  9088,  5565,
        3386])

In [76]:
####EMアルゴリズムで有限混合ロジットモデルを推定####
##完全データのロジットモデルの対数尤度
def cll(b, y, y_vec, Data, zpt, hhpt, member, k, segment, u_id):
    #パラメータの設定
    beta = b.reshape(k, segment)

    #潜在変数での重み付き確率
    U = np.exp(np.sum(np.dot(Data, beta) * zpt[u_id, ], axis=1).reshape(hhpt, member))
    Pr = U / np.sum(U, axis=1).reshape(hhpt, 1)

    #完全データの対数尤度の和
    LL = -np.sum(np.dot((y * np.log(Pr)), np.repeat(1, member)))
    return(LL)

In [3]:
##完全データのロジットモデルの対数微分関数
def dll(b, y, y_vec, Data, zpt, hhpt, member, k, segment, u_id):
    #パラメータの設定
    beta = b.reshape(k, segment)

    #潜在変数での重み付き確率
    U = np.exp(np.dot(Data, beta))
    dlogit = np.zeros((segment, k))

    for j in range(segment):
        #効用と確率を設定
        u = U[:, j].reshape(hhpt, member)
        Pr = u / np.sum(u, axis=1).reshape(hhpt, 1)

        #勾配ベクトルを定義
        Pr_vec = Pr.reshape(-1)
        dlogit[j, :] = np.sum(zpt[u_id, j].reshape(hhpt*member, 1) * (y_vec - Pr_vec).reshape(hhpt*member, 1) * Data, axis=0)

    LLd = -dlogit.T.reshape(-1)
    return(LLd)

In [78]:
##観測データでの尤度と潜在変数zを計算する関数
def ollz(beta, y, Data, zpt, hhpt, member, k, segment, user_dt):
    #潜在変数ごとの確率と尤度を設定
    U = np.exp(np.dot(Data, beta))
    LLho = np.zeros((hh, segment))

    for j in range(segment):
        #効用と確率を設定
        u = U[:, j].reshape(hhpt, member)
        Pr = u / np.sum(u, axis=1).reshape(hhpt, 1)

        #ユーザーごとの尤度を取る
        LLho[:, j] = np.exp((scipy.dot(user_dt, sparse.csr_matrix(np.dot(y * np.log(Pr), np.repeat(1, member))).T).todense())).reshape(-1)

    #観測データの対数尤度
    r = np.tile(theta, hh).reshape(hh, segment)
    LLobz = np.sum(np.log(np.dot(r * LLho, np.repeat(1, segment))))

    #潜在変数zの推定
    LLi = r * LLho
    z = LLi / np.dot(LLi, np.repeat(1, segment)).reshape(hh, 1)
    return LLobz, z

In [79]:
##EMアルゴリズムの設定
iter = 0
rp = 200   #最大繰り返し数
LL = -1000000000   #対数尤度の初期値
dl = 100   #EMアルゴリズムの対数尤度の差の初期値
tol = 0.1
maxit = 20   #準ニュートン法のステップ数

In [80]:
##EMアルゴリズムの初期値を設定
#混合率と潜在変数の初期値
theta = np.repeat(1/segment, segment)
zpt = np.tile(theta, hh).reshape(hh, segment)

#パラメータの初期値
b = np.random.uniform(-0.25, 0.25, segment*k)
beta = b.reshape(k, segment)

In [81]:
##観測データの対数尤度と潜在変数
oll = ollz(beta, y, Data, zpt, hhpt, member, k, segment, user_dt)
zpt = oll[1]
LL1 = oll[0]

In [2]:
dll()

NameError: name 'dll' is not defined

In [82]:
##EMアルゴリズムでパラメータを推定
while abs(dl) >= tol:
    
    #完全データのロジットモデルの推定(Mステップ)
    res = optimize.minimize(cll, b, jac=dll, method='BFGS', args=(y, y_vec, Data, zpt, hhpt, member, k, segment, u_id),
                            options={'gtol': 0.1})
    b = res.x; beta = b.reshape(k, segment)   #パラメータの更新
    theta = np.sum(zpt, axis=0) / hh   #混合率を更新

    #Eステップで対数尤度の期待値と潜在変数を更新(Eステップ)
    obsllz = ollz(beta, y, Data, zpt, hhpt, member, k, segment, user_dt)
    LL = obsllz[0]
    zpt = obsllz[1]

    #EMアルゴリズムのパラメータを更新
    iter = iter + 1
    dl = LL - LL1
    LL1 = LL
    print(LL)

-164518.74317421232
-137671.9776436865
-131752.60106967928
-131581.33994235296
-131576.8270568978
-131575.56276250302
-131575.5627587411


In [83]:
####推定結果と要約####
##推定されたパラメータと真のパラメータの比較
#推定されたパラメータ
pd.DataFrame(np.hstack((beta, betat)))

,0,1,2,3,4,5,6,7,8,9
0,0.442120,0.415331,-0.669589,0.578079,-0.753850,0.289471,0.165273,0.058958,-0.475797,-0.244168
1,0.541735,0.323375,-0.305023,-0.650535,0.404236,-0.552085,-0.033797,0.326540,0.515239,-0.216626
2,0.861292,0.126183,-0.554348,0.335457,-0.329949,-0.099296,0.436373,0.561347,-0.490813,0.192593
3,0.356794,-0.889380,0.398615,0.245464,-0.804896,0.508060,0.279071,-0.589501,-0.469960,-0.399106
4,-0.006914,0.352680,0.472988,-0.716814,0.810233,-0.006988,0.119572,-0.286488,0.583403,-0.457416
5,0.355476,-0.390766,-0.368169,0.110008,-0.409728,0.522521,0.199805,-0.202437,-0.583005,-0.075811
6,0.752743,0.034924,0.253657,0.830790,-0.196782,0.319229,0.425201,0.316186,0.497846,0.520847
7,0.616349,0.066215,-0.036229,0.201823,0.137366,-0.033694,0.214125,-0.295274,0.023562,0.226259
8,0.539408,0.710948,-0.684260,0.328381,0.513074,0.427096,0.187161,0.385816,-0.252715,-0.468122
9,0.996132,2.477183,2.711826,0.412365,1.217749,0.463481,1.052261,2.642467,1.173632,2.794052


In [84]:
##混合率とセグメントへの所属確率
print(np.round(np.vstack((theta, thetat)), 3))   #推定された混合率
np.argmax(zpt, axis=1)   #セグメントへの所属
pd.DataFrame(np.hstack((np.round(zpt, 3), Z)))   #潜在変数の割当確率と真のセグメントを比較

[[0.372 0.075 0.294 0.137 0.122]
 [0.14  0.364 0.076 0.121 0.299]]


,0,1,2,3,4,5,6,7,8,9
0,0.000,0.000,0.000,0.000,1.000,0.0,0.0,0.0,1.0,0.0
1,0.006,0.000,0.000,0.994,0.000,1.0,0.0,0.0,0.0,0.0
2,1.000,0.000,0.000,0.000,0.000,0.0,1.0,0.0,0.0,0.0
3,0.000,0.000,1.000,0.000,0.000,0.0,0.0,0.0,0.0,1.0
4,0.000,0.000,1.000,0.000,0.000,0.0,0.0,0.0,0.0,1.0
5,0.869,0.131,0.000,0.000,0.000,0.0,1.0,0.0,0.0,0.0
6,0.963,0.000,0.000,0.037,0.000,0.0,1.0,0.0,0.0,0.0
7,0.000,0.000,1.000,0.000,0.000,0.0,0.0,0.0,0.0,1.0
8,0.000,0.000,1.000,0.000,0.000,0.0,0.0,0.0,0.0,1.0
9,0.003,0.997,0.000,0.000,0.000,0.0,0.0,1.0,0.0,0.0


In [330]:
##AICとBICの計算
print(np.round(LL, 3))   #最大化された観測データの対数尤度
print(np.round(-2*LL + 2*(k*segment-1), 3))   #AIC
print(np.round(-2*LL + np.log(hhpt)*(k*segment-1), 3))   #BIC

-114552.95
229453.9
231108.846
